#problems
1. The transcript data we have is a bit less to procede with extracting proper topics because all the models expect a bit more data.

---
#solutions
1. Using a countvectorizer to remove stop words(default) and mentioning it in the model declaration seems to have drastically improved the suggestions by the model.



In [70]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd


/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.divide(0.4, 1, casting="unsafe", dtype=np.float),


In [ ]:

 
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.visualize_topics()

In [ ]:
len(docs)

#Using transcripts

In [ ]:
with open("/content/wells fargo doc.txt","r") as f:
  string = f.read()

print(string)


Operator
Welcome, and thank you for joining the Wells Fargo Second Quarter 2022 Earnings Conference Call. All lines have been placed on mute to prevent any background noise. After the speakers' remarks, there will be a question-and-answer session. [Operator Instructions] Please note that today's call is being recorded. I would now like to turn the call over to your host, John Campbell, Director of Investor Relations. Sir, you may begin the conference.

John Campbell
Good morning. Thank you for joining our call today where our CEO, Charlie Scharf; and our CFO, Mike Santomassimo, will discuss second quarter results and answer your questions. This call is being recorded.

Before we get started, I would like to remind you that our second quarter earnings materials, including the release, financial supplement, and presentation deck are available on our website at wellsfargo.com. I'd also like to caution you that we may make forward-looking statements during today's call that are subject to 

In [ ]:
print(len(string))

68243


In [ ]:
clean_string = []

for s in string.split("\n"):
  if s.isspace() or s == "":
    pass
  else:
    clean_string.append(s)
print(clean_string)
print(len(clean_string))
    

['Operator', "Welcome, and thank you for joining the Wells Fargo Second Quarter 2022 Earnings Conference Call. All lines have been placed on mute to prevent any background noise. After the speakers' remarks, there will be a question-and-answer session. [Operator Instructions] Please note that today's call is being recorded. I would now like to turn the call over to your host, John Campbell, Director of Investor Relations. Sir, you may begin the conference.", 'John Campbell', 'Good morning. Thank you for joining our call today where our CEO, Charlie Scharf; and our CFO, Mike Santomassimo, will discuss second quarter results and answer your questions. This call is being recorded.', "Before we get started, I would like to remind you that our second quarter earnings materials, including the release, financial supplement, and presentation deck are available on our website at wellsfargo.com. I'd also like to caution you that we may make forward-looking statements during today's call that are

In [ ]:
topic_model_wells_fargo = BERTopic()
topics_custom, probs_custom = topic_model_wells_fargo.fit_transform(clean_string)

In [ ]:
topic_model_wells_fargo.get_topic_info()

,Topic,Count,Name
0,-1,20,-1_juneja_vivek_ebrahim_poonawala
1,0,208,0_the_and_to_of
2,1,39,1_thanks_good_thank_morning
3,2,23,2_charlie_scharf_yeah_hey
4,3,21,3_santomassimo_mike__
5,4,18,4_betsy_graseck_question_thank
6,5,15,5_operator_jefferies_questionandanswer_begin
7,6,12,6_siefers_scott_matt_oconnor
8,7,11,7_open_line_your_is
9,8,10,8_john_mcdonald_chubak_steven


In [ ]:
topic_model_wells_fargo.visualize_topics()

initial analyses of just passing the lines seperated by the line breaker doesnt seems to produce noticible results


In [ ]:
with open("/content/names.txt","r") as f:
  name_string = f.read()


In [ ]:
name_string

"John Campbell\tDirector Investor Relations\nCharlie Scharf\tChief Executive Officer\nMike Santomassimo\tChief Financial Officer\nKen Usdin\tJefferies\nJohn McDonald\tAutonomous Research\nSteven Chubak\tWolfe Research\nScott Siefers\tPiper Sandler\nJohn Pancari\tEvercore ISI\nErika Najarian\tUBS\nEbrahim Poonawala\tBank of America\nGerard Cassidy\tRBC Capital Markets\nBetsy Graseck\tMorgan Stanley\nVivek Juneja\tJPMorgan\nMatt O'Connor\tDeutsche Bank"

In [ ]:
cols = ["name","position"]
name_df = pd.read_csv("/content/names.txt",sep="\t",names=cols)

,name,position
0,John Campbell,Director Investor Relations
1,Charlie Scharf,Chief Executive Officer
2,Mike Santomassimo,Chief Financial Officer
3,Ken Usdin,Jefferies
4,John McDonald,Autonomous Research
5,Steven Chubak,Wolfe Research
6,Scott Siefers,Piper Sandler
7,John Pancari,Evercore ISI
8,Erika Najarian,UBS
9,Ebrahim Poonawala,Bank of America


In [ ]:
#trying a new clean function
#since before each paragraph a persons name is mentioned so using that as a reference point
chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>30:
      temp_dict[temp_name].append(i)
  

In [ ]:
temp_dict["Charlie Scharf"]

["Thanks very much, John. Good morning. I'll make some brief comments about our second quarter results, the operating environment, and update you on our priorities. I'll then turn the call over to Mike to review second quarter results in more detail before we take your questions.",
 'Let me start with some second quarter highlights. We earned $3.1 billion in the second quarter, our results included a $576 million impairment of equity securities, predominantly in our venture capital business due to market conditions. Revenue declined as growth in net interest income driven by rising interest rates and higher loan balances was more than offset by lower non-interest income as market conditions negatively impacted our venture capital, mortgage banking, investment banking, and wealth management advisory businesses.',
 'We continued to execute on our efficiency initiatives and our expenses declined from a year ago even with inflationary pressures and higher operating losses. We had broad-bas

In [ ]:


vectorizer_model = CountVectorizer( stop_words="english")

In [ ]:
custom_doc = temp_dict["John Campbell"]+temp_dict["John McDonald"]+temp_dict["Mike Santomassimo"]+temp_dict["Charlie Scharf"]

In [ ]:
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,33,-1_year_new_credit_card
1,0,117,0_quarter_year_think_loan
2,1,28,1_business_capabilities_fargo_wells


In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
topic_model_custom.visualize_topics()

#APPle transcripts

In [ ]:
with open("/content/apple_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("/content/apple_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>30:
      temp_dict[temp_name].append(i)

In [ ]:
custom_doc = temp_dict["Timothy Cook"] +temp_dict["Luca Maestri"]

In [ ]:
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)

topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,49,-1_quarter_world_june_macroeconomic
1,0,19,0_new_customers_apple_developers
2,1,15,1_quarter_june_billion_revenue
3,2,13,2_foreign_exchange_impact_basis
4,3,12,3_quarter_guidance_think_number


In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
topic_model_custom.visualize_topics()

In [ ]:
custom_doc = temp_dict["Luca Maestri"]
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(custom_doc)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,57,-1_quarter_june_billion_services


In [ ]:
topic_model_custom.visualize_barchart()

In [ ]:
#collective data 
full_conversation = []
for i in temp_dict.values():
  full_conversation+=i
topic_model_custom = BERTopic(vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model_custom.fit_transform(full_conversation)

In [ ]:
topic_model_custom.get_topic_info()

,Topic,Count,Name
0,-1,32,-1_question_thank_good_like
1,0,73,0_quarter_think_business_impact
2,1,55,1_new_apple_quarter_revenue


In [ ]:
topic_model_custom.visualize_barchart()

#trying top2Vec


In [ ]:
pip install top2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from top2vec import Top2Vec

model = Top2Vec(temp_dict["Mike Santomassimo"]+temp_dict["Charlie Scharf"]+temp_dict["John Campbell"]+temp_dict["John McDonald"])

2022-07-30 13:31:08,557 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2022-07-30 13:31:08,651 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2022-07-30 13:31:09,301 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2022-07-30 13:31:13,583 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2022-07-30 13:31:13,601 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


ValueError: ignored

In [ ]:
model.get_topics()

# using transformers


In [66]:
pip install bertopic[flair]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.0 MB/s 
     |████████████████████████████████| 5.2 MB 10.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 6.4 MB/s 
     |████████████████████████████████| 88 kB 9.6 MB/s 
     |████████████████████████████████| 4.7 MB 45.2 MB/s 
     |████████████████████████████████| 401 kB 63.8 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 1.6 MB 67.4 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 981 kB 63.2 MB/s 
     |████████████████████████████████| 1.2 MB 72.1 MB/s 
     |████████████████████████████████| 788 kB 68.9 MB/s 
     |█████████████████████████

In [67]:
from flair.embeddings import TransformerDocumentEmbeddings

In [ ]:
roberta = TransformerDocumentEmbeddings('roberta-base')
#vectorizer_model=vectorizer_model
topic_model = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model)

In [ ]:
with open("/content/apple_transcript.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("/content/apple_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>30:
      temp_dict[temp_name].append(i)

In [ ]:
custom_doc = temp_dict["Timothy Cook"] +temp_dict["Luca Maestri"]

In [ ]:
topics_custom, probs_custom = topic_model.fit_transform(custom_doc)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,14,-1_year_quarter_revenue_5g
1,0,41,0_expect_quarter_developers_different
2,1,30,1_quarter_think_ve_impact
3,2,23,2_new_apple_billion_quarter


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:
#usuing finbert

finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
#vectorizer_model=vectorizer_model
topic_model = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)

In [ ]:
custom_doc = temp_dict["Timothy Cook"]

In [ ]:
topics_custom, probs_custom = topic_model.fit_transform(custom_doc)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,40,0_apple_new_quarter_customers
1,1,13,1_quarter_june_saw_macroeconomic


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:
#collective data 
full_conversation = []
for i in temp_dict.values():
  full_conversation+=i


In [ ]:
topic_model = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model)


In [ ]:
topics_custom, probs_custom = topic_model.fit_transform(full_conversation)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,35,-1_billion_quarter_june_year
1,0,63,0_apple_think_new_iphone
2,1,29,1_impact_quarter_saw_sort
3,2,17,2_quarter_june_revenue_record
4,3,16,3_question_ll_operator_today


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model)
topics_custom, probs_custom = topic_model.fit_transform(full_conversation)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,31,-1_ll_today_question_iphone
1,0,51,0_expect_quarter_product_luca
2,1,34,1_quarter_look_think_year
3,2,23,2_new_apple_billion_quarter
4,3,21,3_sort_just_maybe_follow


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

# CISCO DATA

In [65]:
with open("/content/cisco_transcripts.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("/content/cisco_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>15:
      temp_dict[temp_name].append(i)

In [74]:
temp_dict["Chuck Robbins"]

['Thanks, Marilyn, and good afternoon, everyone. When we spoke with you back in February, we entered Q3 in the second half of our fiscal year with optimism despite the supply and component challenges and other headwinds impacting us and many of our peers. Many of those factors that fueled that optimism remain unchanged today.',
 'We continue to see strong demand resulting in record backlog, our business transformation is progressing well and our differentiated innovation across our portfolio is helping our customers embrace and adopt the multiple technology transitions happening.',
 'However, there were two unanticipated events since our last earnings call, which impacted our Q3 revenue performance.',
 'The first is the war in Ukraine. This resulted in us ceasing operations in Russia and Belarus and had a corresponding revenue impact, which Scott will discuss.',
 'The second relates to COVID related lockdowns in China, which began in late March. These lockdowns resulted in an even more

In [ ]:
seed_topic_list = [["covid", "war", "supply", "impact"],["revenue","margins","customer","growth"]]

In [114]:
roberta = TransformerDocumentEmbeddings('roberta-base')
#vectorizer_model=vectorizer_model
vectorizer_model = CountVectorizer( stop_words="english",token_pattern='[a-zA-Z0-9]{3,}' )
topic_model = BERTopic(embedding_model=roberta,vectorizer_model=vectorizer_model,top_n_words = 10,n_gram_range =(1,2))

In [115]:
custom_data = temp_dict["Chuck Robbins"] + temp_dict["Marilyn Mora"] + temp_dict["Scott Herren"]

In [ ]:
topics_custom, probs_custom = topic_model.fit_transform(custom_data)

In [ ]:
topic_model.get_topic_info()

In [106]:
topic_model.get_topic(1)

[('think', 0.08215372138435002),
 ('quarter', 0.06532005488801017),
 ('just', 0.056484580204199185),
 ('going', 0.0538721894902766),
 ('right', 0.05065302586477297),
 ('open', 0.03954792845346211),
 ('yes', 0.03781366790955741),
 ('know', 0.03460443739677935),
 ('talked', 0.03460443739677935),
 ('time', 0.03315211660940098),
 ('don', 0.033095476932844155),
 ('way', 0.03244235900426712),
 ('day', 0.0312845901308908),
 ('growth', 0.03123575270135289),
 ('revenue', 0.030486132991395062)]

In [113]:
topic_model.visualize_barchart()

In [108]:
topic_model.visualize_topics()

/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1595: RuntimeWarning:

k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.



TypeError: ignored

In [ ]:
#usuing finbert

finbert = TransformerDocumentEmbeddings('ProsusAI/finbert')
#vectorizer_model=vectorizer_model
topic_model = BERTopic(embedding_model=finbert,vectorizer_model=vectorizer_model,seed_topic_list=seed_topic_list)

In [ ]:
topics_custom, probs_custom = topic_model.fit_transform(custom_data)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6,-1_revenue_total_growth_product
1,0,122,0_think_quarter_just_revenue
2,1,42,1_year_growth_business_demand


In [ ]:
topic_model.visualize_barchart()

#LDA Analysis 
* will perform lda analysis on cisco data

In [1]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=70e1f99bcd868b7b27ad0dd95b844a8a04d013b37c87e3a65136de7b490e9ebf
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [2]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [3]:
import pandas as pd

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
import numpy as np
np.random.seed(2018)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
nltk.download("omw-1.4")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:
stemmer = PorterStemmer()

In [9]:
with open("/content/cisco_transcripts.txt","r") as f:
  string = f.read().split("\n")
  clean_string = []
  for s in string:
    if s.isspace() or s == "":
      pass
    else:
      clean_string.append(s)

cols = ["name","position"]
name_df = pd.read_csv("/content/cisco_names.txt",sep="\t",names=cols)

chat_list = []
temp_name = ""
temp_dict = {}
user_names = list(name_df.name)+["Operator"]
for i in clean_string:
  if i in user_names and i!="":
    temp_name = i
    if i not in temp_dict:  
      temp_dict[i]= []
  else:
    if len(i)>20:
      temp_dict[temp_name].append(i)

###observation 
* dividing the dialogs on the basis of its orginal form is better since combining all the dialogs seems to create worst results


In [10]:
temp_dict["Chuck Robbins"]

['Thanks, Marilyn, and good afternoon, everyone. When we spoke with you back in February, we entered Q3 in the second half of our fiscal year with optimism despite the supply and component challenges and other headwinds impacting us and many of our peers. Many of those factors that fueled that optimism remain unchanged today.',
 'We continue to see strong demand resulting in record backlog, our business transformation is progressing well and our differentiated innovation across our portfolio is helping our customers embrace and adopt the multiple technology transitions happening.',
 'However, there were two unanticipated events since our last earnings call, which impacted our Q3 revenue performance.',
 'The first is the war in Ukraine. This resulted in us ceasing operations in Russia and Belarus and had a corresponding revenue impact, which Scott will discuss.',
 'The second relates to COVID related lockdowns in China, which began in late March. These lockdowns resulted in an even more

In [11]:
custom_data = temp_dict["Chuck Robbins"] + temp_dict["Marilyn Mora"]+temp_dict["Scott Herren"]

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(custom_data))
print(data_words[2])

['however', 'there', 'were', 'two', 'unanticipated', 'events', 'since', 'our', 'last', 'earnings', 'call', 'which', 'impacted', 'our', 'revenue', 'performance']


In [13]:
import spacy

In [ ]:
!python3 -m spacy download en

In [14]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [15]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [16]:
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN","ADJ"]) #select noun and verb
print(data_lemmatized[:2])

['thank good afternoon second half fiscal year optimism supply component challenge other headwind many peer many factor optimism unchanged today', 'strong demand record backlog business transformation differentiated innovation portfolio customer multiple technology transition']


In [17]:
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize      
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
vectorizer_tfidf = TfidfVectorizer()#dangerous

In [19]:
#ngram (1,3)
vectorizer = CountVectorizer(analyzer='word',                                
                             min_df=10,

# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}'  
# num chars > 3
                             # max_features=50000,             
# max number of uniq words    
)
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [48]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=15,               
# Max learning iterations
                                      learning_method='batch',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)

LatentDirichletAllocation(max_iter=15, n_jobs=-1, random_state=100)


In [49]:
lda_output

array([[0.02000104, 0.02      , 0.02000083, ..., 0.81997055, 0.02000526,
        0.02001359],
       [0.01428732, 0.01428648, 0.01428625, ..., 0.01428571, 0.01428585,
        0.57877954],
       [0.05      , 0.05      , 0.05001399, ..., 0.05      , 0.05      ,
        0.05      ],
       ...,
       [0.02      , 0.02000673, 0.22000003, ..., 0.02000073, 0.61998427,
        0.02000061],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05000828,
        0.54998672],
       [0.05      , 0.05      , 0.55      , ..., 0.05      , 0.05      ,
        0.05      ]])

In [50]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2463.9717030867414
Perplexity:  38.28003783127336
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [58]:
# Define Search Param
search_params = {'n_components': [ 5, 10, 15, 20, 25, 30], 'learning_decay': [.4, .5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='batch', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=15,
             mean_change_tol=0.001, n_components=None, n_jobs=1,
      perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
       param_grid={'n_components': [5,10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

GridSearchCV(error_score='raise',
             estimator=LatentDirichletAllocation(learning_method=None,
                                                 max_iter=15, n_components=None,
                                                 n_jobs=1),
             n_jobs=1,
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 10, 15, 20, 25, 30]},
             return_train_score='warn')

In [59]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.4, 'n_components': 5}
Best Log Likelihood Score:  -638.4655157377955
Model Perplexity:  37.60799500301649


In [60]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic"+ str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(custom_data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.040000,0.040000,0.040000,0.400000,0.480000,4
Doc1,0.340000,0.570000,0.030000,0.030000,0.030000,1
Doc2,0.600000,0.100000,0.100000,0.100000,0.100000,0
Doc3,0.730000,0.070000,0.070000,0.070000,0.070000,0
Doc4,0.050000,0.050000,0.800000,0.050000,0.050000,2
Doc5,0.530000,0.050000,0.050000,0.050000,0.310000,0
Doc6,0.120000,0.130000,0.030000,0.550000,0.170000,3
Doc7,0.040000,0.040000,0.210000,0.040000,0.670000,4
Doc8,0.100000,0.100000,0.100000,0.600000,0.100000,3
Doc9,0.370000,0.180000,0.080000,0.020000,0.360000,0


In [61]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,backlog,basis,business,chain,component,customer,demand,enterprise,good,growth,...,solid,strong,supply,team,term,thing,time,way,week,year
Topic0,17.953126,4.370770,4.532611,1.229756,0.200245,0.201941,2.394677,0.202740,0.200625,26.040001,...,6.819866,2.231446,0.201048,0.200024,1.254635,0.203938,0.200976,0.201943,8.296933,13.775827
Topic1,0.202176,0.202105,9.364425,0.200484,0.200112,4.771128,18.712452,0.345982,5.558917,0.898184,...,3.577977,8.105083,0.200596,0.201111,0.200390,0.271031,0.286196,7.199526,0.200025,2.530601
Topic2,0.200023,10.023756,4.205231,0.202442,9.574689,24.613842,0.200980,15.049877,0.212499,0.213123,...,0.200811,1.255650,1.404049,5.684033,2.789821,9.782246,0.200706,0.202157,2.451945,1.480756
Topic3,0.201612,0.200904,0.200399,10.166998,8.202029,0.202287,1.248421,0.201308,0.201426,4.813188,...,0.200484,1.205411,28.208632,2.836925,0.204673,4.230462,19.106194,5.166622,3.848362,3.305443
Topic4,0.443063,0.202464,2.697334,0.200320,1.822925,0.210803,0.443470,0.200094,5.826533,5.035504,...,0.200861,0.202410,0.985676,5.077908,12.550481,5.512322,1.205928,0.229751,0.202735,33.907374


In [62]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=5):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=5)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4
Topic 0,revenue,growth,product,backlog,software
Topic 1,question,demand,business,strong,way
Topic 2,customer,enterprise,product,basis,thing
Topic 3,quarter,supply,time,chain,component
Topic 4,year,guidance,term,quarter,good


In [63]:
import pyLDAvis
import pyLDAvis.sklearn

In [64]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.201933 -0.090046       1        1  30.480209
3     -0.060921  0.227056       2        1  20.155355
2      0.102239 -0.073549       3        1  18.007547
4      0.006116  0.092806       4        1  17.886652
1     -0.249368 -0.156266       5        1  13.470237, topic_info=        Term       Freq      Total Category  logprob  loglift
31      year  54.000000  54.000000  Default  30.0000  30.0000
19  question  21.000000  21.000000  Default  29.0000  29.0000
5   customer  28.000000  28.000000  Default  28.0000  28.0000
20   revenue  39.000000  39.000000  Default  27.0000  27.0000
6     demand  21.000000  21.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
15     point   0.192815  21.366744   Topic5  -6.1575  -2.7032
11    impact   0.191877  23.075092   Topic5  -6.1624  -2.7850
24    supply   0.191686  27.715794   Topic5  -6.1634  -2.9692
20   revenue   0.191594  39.648512   Topic5  -6.1639  -3.3277
17   product   0.191589  33.635468   Topic5  -6.1639  -3.1633

[190 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.921570   backlog
1         1  0.279727     basis
1         3  0.629385     basis
2         1  0.197356  business
2         3  0.197356  business
...     ...       ...       ...
31        1  0.236879      year
31        2  0.054664      year
31        3  0.018221      year
31        4  0.637752      year
31        5  0.036443      year

[87 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])